In [81]:
import csv
import re
import numpy as np
from pathlib import Path
from BAG_Code_tw520.BayesianAttackGraph import parse_dot, evidences_to_string
from BAG_Code_tw520.createANDtable import create_AND_table
from BAG_Code_tw520.createORtable import create_OR_table
from BAG_Code_tw520.Tools_tree import kts_layer_static, kts_layer_original
import BAG_Code_tw520.Loopy as Loopy

from pgmpy.inference.ExactInference import BeliefPropagation
from pgmpy.factors.discrete import TabularCPD

In [82]:
# Name of the simulation
simulation = "HighLevel"

# Constante for probability one
ONE = 0.999

In [83]:
# Path to the folder containing the tree
path = Path.cwd() / ("Personnal_simulations/output_" + simulation + "/strongly_connected_components/")
basename = "comparison"

# We all read from the file the edges and the nodes
BAG_static, null, nodes_static = parse_dot(open(Path.cwd() / ("Personnal_simulations/output_" + simulation + "_no_HP/strongly_connected_components/" + "ag-nocycles.dot"), 'r').read(), ONE)
BAG_dynamic, null, nodes_dynamic = parse_dot(open(path /"ag-nocycles.dot", 'r').read(), ONE)

# We create a dictionary to get the node number from the label
kts_layer_static(BAG_static, ONE, nodes_static)
kts_layer_original(BAG_dynamic, ONE, nodes_dynamic)

In [84]:
def deviation(BAG_static, BAG_dynamic, nodes_static, evidences, calculate_probabilities=False):
    for type in evidences.keys():
        evidences[type] = {str(k): 1 for k in evidences[type]}
    # Exact inference
    if calculate_probabilities ==1:
        nodes_wo_evidences = [n for n in nodes_static if n not in evidences['static'].keys()]
        nodes_1 = nodes_wo_evidences[:len(nodes_wo_evidences)//3]
        nodes_2 = nodes_wo_evidences[len(nodes_wo_evidences)//3:(len(nodes_wo_evidences)*2)//3]
        nodes_3 = nodes_wo_evidences[(len(nodes_wo_evidences)*2)//3:]
        prop = {}
        prop['static'] = BeliefPropagation(BAG_static)
        prop['dynamic'] = BeliefPropagation(BAG_dynamic)
        total_prob1 = {}
        total_prob2 = {}
        total_prob3 = {}
        for p in prop.items():
            total_prob1[p[0]] = p[1].query(nodes_1, evidence=evidences[p[0]])
            total_prob2[p[0]] = p[1].query(nodes_2, evidence=evidences[p[0]])
            total_prob3[p[0]] = p[1].query(nodes_3, evidence=evidences[p[0]])

    def probability(node, type):
        prob = 1
        if node in evidences.keys():
            prob = evidences[node]
        else:
            if node in nodes_1:
                prob = total_prob1[type].marginalize([n for n in nodes_1 if n != node], inplace=False).values[1]
            elif node in nodes_2:
                prob = total_prob2[type].marginalize([n for n in nodes_2 if n != node], inplace=False).values[1]
            elif node in nodes_3:
                prob = total_prob3[type].marginalize([n for n in nodes_3 if n != node], inplace=False).values[1]
        return prob
    
    # Display the nodes
    prob_static = []
    prob_dynamic = []
    for node in nodes_static.keys():
            prob_static.append(probability(node, 'static'))
            prob_dynamic.append(probability(node, 'dynamic'))
    return prob_static, prob_dynamic

In [85]:
def variation(prob_static, prob_dynamic):
    sum = 0
    lenght = 0
    for i in range(len(prob_static)):
        if prob_static[i] <0.9:
            dif = 2 * abs(prob_dynamic[i] - prob_static[i])/(prob_static[i] + prob_dynamic[i])
            sum += dif
            lenght += 1
    return sum/lenght

def difference(prob_static, prob_dynamic):
    sum = 0
    lenght = 0
    for i in range(len(prob_static)):
        if prob_static[i] <0.9:
            dif = abs(prob_dynamic[i] - prob_static[i])
            sum += 100*dif
            lenght += 1
    return sum/lenght
    

In [87]:
evidences_static = [[11],[11,9], [11,9,7, 5]]
evidences_dynamic = [[11],[11,9], [11,9,7, 19]]
for i in range(1, 4):
    prob_static, prob_dynamic = deviation(BAG_static, BAG_dynamic, nodes_static, {'static' : evidences_static[i-1], 'dynamic' : evidences_dynamic[i-1]}, calculate_probabilities=1)
    print("Step ", i, " : ", prob_static, prob_dynamic)
    var = variation(prob_static, prob_dynamic)
    dif = difference(prob_static, prob_dynamic)
    print("     \\hline Step ", i, " & ", "{:.1f}".format(dif), "\\% & ", int(var * 100), "\\% \\\\")


c:\Users\docuser\AppData\Local\Programs\Python\Python312\Lib\site-packages\pgmpy\factors\discrete\DiscreteFactor.py:796: RuntimeWarning: invalid value encountered in divide
  phi.values = phi.values / phi1.values


Step  1  :  [0.0010867004617258003, 9.644100303203507e-05, 0.0013075682389505274, 0.0003421226239716644, 0.008617698336817744, 0.007216919357842707, 0.016489031616346884, 0.017229178661120004, 0.03936478399999999, 0.04801599999999999, 1, 0.0013980468343650796, 0.002746869762584643, 0.0021863201033600023] [0.006279955285008553, 0.0058731426974511156, 0.007796737043980229, 0.007560330415995804, 0.03556497079120596, 0.030123258365595994, 0.026821970106756046, 0.028722992332320407, 0.03482291988511434, 0.04233156431178265, 1, 0.013478012045664076, 0.013767407102124945, 0.015979232918804688]
     \hline Step  1  &  1.1 \% &  113 \% \\
Step  2  :  [0.0014774913006525836, 0.0005311360407703928, 0.007201258755496407, 0.0068979518971039, 0.17375193695475816, 0.1726535203776001, 0.39447432000000016, 0.4376800000000001, 1, 0.9758163540285144, 1, 0.023094803081600015, 0.04537646000000005, 0.05554000000000006] [0.1297474027668557, 0.14321179395645792, 0.18159753308653925, 0.20088713357790794, 0.644